Hey Luke, 

 You may want to consult with Matt briefly on how to use the fluoresence microscope with the GigaCE commands. In particular, you will want to record what excitation channel he is using ('cyan', 'blue', etc...) and what emission channel he is using (1, 2, 3, etc...). Although I think the defaults I have chosen are correct. 
 
 Because the Nikton TE300 has a very limited field of view, it may not seem right to you. I would watch Matt use the fluorescence capabilities first so you have an idea of what is 'normal'. 
 
 The following code was intended for you to run at the end of the day, after you single-cell CE experiments. 

# Step 1, Load the Modules

In [3]:
import os
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
    
from L4 import Focus
from importlib import reload
%config Completer.use_jedi = False

# Basic Function Modules
import sys
import os
import glob
from imp import reload
import csv
import threading
from pathlib import Path
import warnings
import logging

#Data Analysis Modules
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Image Detection Modules
import skimage.measure as measure
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
#AutomatedCE Modules
import testing.inputs  
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO
from L4 import Focus
from L4 import AutomatedControl
from L4 import Trajectory
from L4.image_util import ImageSaver
from L3 import SystemsBuilder
#import testing.te300_auto_utils as auto_utils
from L4 import image_util
from L4 import FileIO


try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    import sys
    !conda install --yes --prefix {sys.prefix} tqdm
    from tqdm.notebook import tqdm


# Reload (when changing aspects in the following modules)
reload(Focus)
reload(DisplayWindows)
reload(testing.inputs)

# Make the plots pop up in a separate window
%matplotlib qt



def te300_presnap(ce_sys, intensity=1):
    ce_sys.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def te300_postsnap(ce_sys, intensity=0):
    ce_sys.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def pre_fluoresence(ce_system,rgb='B',bins=2,fluor_channel=4,
                    wait=0.1, exp=500,light_channel='cyan', auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    ce_system.excitation_wheel.set_channel([light_channel])
    ce_system.camera.set_exposure(exp)
    ce_system.camera.set_binning(bins)
    if not auto_shutter:
        ce_system.camera._presnap_callbacks=[]
        ce_system.camera._postsnap_callbacks=[]
        ce_system.camera.add_presnap_callback(te300_presnap, ce_sys=ce_system)
        ce_system.camera.add_postsnap_callback(te300_postsnap, ce_sys=ce_system)
    time.sleep(wait)
    
def post_fluoresence(ce_system,rgb_channel='B', old_exp=10,
                     old_bin=1,old_channel=1,
                     auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(old_exp)
    ce_system.filter_wheel.set_channel(old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_channel)
    ce_system.excitation_wheel.set_channel([])
    ce_system.camera.set_binning(old_bin)
    ce_system.camera.continuous_snap()
    if not auto_shutter:
        ce_system.camera._presnap_callbacks=[]
        ce_system.camera._postsnap_callbacks=[]


new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE


C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\MicroControlServer.py


# Step 2 Load the correct config file and open the System

In [4]:
#CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
#CONFIG_PATH = r"D:\Scripts\CEInstrument\AutomatedCE\config\Test-System.cfg"
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\config\TE300.cfg"

reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

ce_system.filter_wheel.set_channel(1)
ce_system.camera.set_exposure(100)


# Adjust the poorly wired RGB lighting
ce_system.inlet_rgb.turn_on_channel('B')
ce_system.inlet_rgb.turn_off_channel('R')

daq
newwave
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x00000203B59F0400>, 'xy_stage': <L2.XYControl.PriorXY object at 0x00000203B59F0460>, 'objective': <L2.ZControl.PriorZ object at 0x00000203B59F04C0>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x00000203B59F04F0>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x00000203B59F0490>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x00000203B59F0580>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000203B59F0550>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000203B59F05E0>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x00000203B59F0A00>, 'lysis_laser': <L2.LaserControl.NewWaveBNC object at 0x00000203B59F0C70>, 'excitation_wheel': <L2.FilterWheelControl.LumencorFilter object at 0x00000203B59F0C40>, 'filter_wheel': <L2.FilterWheelControl.PriorFilter object at 0x00000203B59F0BE0>}
OPENING:  ard1 <L1.Controllers.ArduinoController object at 0x00000203B

'R:Off, G:Off, B:On'

In [5]:

ce_system.filter_wheel.set_channel(1)
ce_system.camera.set_exposure(100)

True

## Test the Imaging 

Below are some of the settings used for brightfield and fluoresence. 

Change data_dir to a new empty folder. This will be the data I need to see how it performed. Once the experiment is done, load that into your UW google drive and share the folder with me :D 

Change the bright field and fluor parameters as needed to get a good image. Whatever is entered here, will be used later on. 


In [6]:
data_dir = r"D:\Luke Focus Images 20210430"
output_csv = os.path.join(data_dir,'simple_data_out.csv')

bright_field_exposure = 12 # in milliseconds
bright_field_bins = 1 # '1', '2', '4', '8' are acceptable
bright_field_emission_channel = 1 # Which emission channel to select
rgb_channel = 'B' # on te300 'B' is red. :D 

fluor_channel = 4
fluor_exposure = 500 # in milliseconds
fluor_bin = 4
fluor_light = 'cyan'


wait = 0.15

pre_fluor_settings = {'rgb':rgb_channel, 'bins':fluor_bin, 'fluor_channel':fluor_channel,
                    'wait':wait, 'exp':fluor_exposure, 'light_channel':fluor_light,
                  'auto_shutter':False}

post_fluor_settings = {'rgb_channel':rgb_channel, 'old_exp':bright_field_exposure,
                     'old_bin':bright_field_bins, 'old_channel':bright_field_emission_channel,
                     'auto_shutter':False}


ce_system.camera.set_exposure(bright_field_exposure)
ce_system.camera.set_binning(bright_field_bins)


### Display the bright field camera output

Make sure the bright_field_exposure, bright field bins, emission channel, and RGB channel are correct. 

In [176]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


### Display the fluoresence camera output

Make sure that the fluor channel, light channel, exposure, and binning are adequate to view your cells. 


In [6]:
resp = get_yes_no("Adjust the mirror for fluoresence")
if not resp:
    raise ValueError(" Put the mirror down for fluoresence")
pre_fluoresence(ce_system, **pre_fluor_settings)
img = ce_system.camera.snap()
post_fluoresence(ce_system, **post_fluor_settings)
plt.imshow(img)

ax = plt.gca()
fig, ax2 = plt.subplots()
ax2.imshow(img)

ax.imshow(img)


Adjust the mirror for fluoresence y
HEYYOO


# More functions we need

In [8]:
# These are functions we will be using. 
def save_me_data(imgs, z_vals, folder, prefix):
    for i, z in zip(imgs, z_vals):
        filename=prefix + f"_{int(z*1e6):08d}.tif"
        io.imsave(os.path.join(folder,filename), i)
        
def save_a_note(prefix, msg, val, file):
    with open(file,'a') as fout:
        fout.write(f"{prefix},{msg},{val}\n")

def gather_data_above_and_below(center, distance=0.1, resolution=0.1, wait=0.15):
    imgs = ImageSaver(ce_system)
    imgs.add_callback()
    
    distances = np.arange(center-distance, center+distance, resolution)
    print(f"From {center-distance} to {center+distance} mm at {resolution} mm resolution")
    for z in tqdm(distances):
        ce_system.objective.set_z(z)
        time.sleep(wait)
        ce_system.camera.snap()
    imgs.remove_callback()
    return distances, imgs.images


# Step 3 Focus Cells near the center of the Sample Chamber

Use the objective that is being used for single-cell analysis (may be 60x or 40x)
The program will randomly sample locations in a 1 CM Diameter from this point, so make sure the center is at least 0.5 cm from the rubber o-ring. 

# Gather Data for Focusing

We are going to try both brightfield and fluoresence focusing. The next few boxes of code will do the following tasks:

1. Record the brightfield data 100 um above and a 100 um below your defined start point. 
2. Run the Focus Climb Algorithm and record the focus point determined by my program. 
3. Change to fluoresence view and repeat: 
4. Record the fluoresence data 100 um above and a 100 um below the defined start point
5. Run the focus algorithm and record the focus point determined by my program 

6. It will test whether cavitation bubbles will form at the Bright field, and fluoresence points. 
7. If lysis did not occur, you will adjust focal position until cavitation/lysis does occur. 



### Gather Bright Field Data

before you run please make sure the cells are in focus (to your best judgement)

In [263]:
center_xy = ce_system.xy_stage.read_xy()
center_z =center = ce_system.objective.read_z()

# Set up the camera for bright field
ce_system.camera.stop()
ce_system.camera.set_exposure(bright_field_exposure)
ce_system.camera.set_binning(bright_field_bins)

xy = ce_system.xy_stage.read_xy()
prefix = f'BF center_{int(center*1e6):08d} x_{int(xy[0]*1e4):04d} y_{int(xy[1]*1e4):04d} '

for dist, res in zip([0.1,0.015, 0.005], [0.01,0.001,0.0002]):
    z_vals, imgs = gather_data_above_and_below(center_z, distance=dist, resolution=res, wait=0)
    print("Saving Data...")
    save_me_data(imgs, z_vals, data_dir, prefix)
ce_system.objective.set_z(center_z)

print('Starting Climb Focus...')

ce_system.objective.set_z(center_z-0.012)
imgs = ImageSaver(ce_system)
imgs.add_callback()
rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
bf_climb_focus = rsc.climb(initial_step = 0.005, final_step=0.0002, refinements=3)
imgs.remove_callback()
cam.live_image()
print("Finished")

From -0.1328 to 0.06720000000000001 mm at 0.01 mm resolution


  0%|          | 0/20 [00:00<?, ?it/s]

Saving Data...


<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0132800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0122800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0112800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0102800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0092800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

From -0.0478 to -0.017800000000000003 mm at 0.001 mm resolution


...

<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0026799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0025799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0024799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0023799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0022799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

  0%|          | 0/50 [00:00<?, ?it/s]

Saving Data...


<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0037800.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0037600.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0037400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0037200.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-8-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images 20210430\BF center_-0032800 x_-266850 y_353680 _-0037000.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

Starting Climb Focus...
OH NO ['R']
HEYYOO
Finished


## Gather Fluoresence Data

In [323]:
resp = get_yes_no("Please flip the mirror for fluoresence.\n Has the mirror been flipped down?")
if not resp:
    raise ValueError('Please flip the mirror!')

Please flip the mirror for fluoresence.
 Has the mirror been flipped down? y


In [324]:
pre_fluoresence(ce_system, **pre_fluor_settings)

prefix = f'FL center_{int(center*1e6):08d} x_{int(xy[0]*1e4):04d} y_{int(xy[1]*1e4):04d} '
for dist, res in zip([0.1,0.015, 0.005], [0.01,0.001,0.0002]):
    z_vals, imgs = gather_data_above_and_below(center_z, distance=dist, resolution=res)
    print("Saving Data...")
    save_me_data(imgs, z_vals, data_dir, prefix)
    
ce_system.objective.set_z(center_z)

print('Starting Climb focus...')
time.sleep(0.75)

# Start the hill climb from 12 microns below focus
ce_system.objective.set_z(center_z-0.012)
imgs = ImageSaver(ce_system)
imgs.add_callback()
rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)

fl_climb_focus = rsc.climb(initial_step = 0.005, final_step=0.0002, refinements=3)
imgs.remove_callback()
cam.live_image()
post_fluoresence(ce_system, **post_fluor_settings)
print("Finished")

From -4.9924 to -4.792400000000001 mm at 0.01 mm resolution


  0%|          | 0/20 [00:00<?, ?it/s]

Saving Data...
From -4.9074 to -4.877400000000001 mm at 0.001 mm resolution


<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4992400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4982400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4972400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4962400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4952400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

  0%|          | 0/30 [00:00<?, ?it/s]

...

<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4907400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4906400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4905399.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4904399.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4903399.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

  0%|          | 0/50 [00:00<?, ?it/s]

Saving Data...


<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4897400.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4897200.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4896999.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4896799.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipython-input-7-92ba4a633ee5>:5: UserWarning: D:\Luke Focus Images\20210222\FL center_-4892400 x_-237440 y_350959 _-4896599.tif is a low contrast image
  io.imsave(os.path.join(folder,filename), i)
<ipyt

Starting Climb focus...
OH NO ['R']
HEYYOO
Finished


# Prepare for Lysis

In [136]:
resp = get_yes_no("Please flip back in preparation for lysis. \n Has the mirror been flipped up?")
if not resp:
    raise ValueError("Please Flip Mirror!")

resp = get_yes_no("Please shut the curtains surrounding the system. \n Have the curtains been closed?")
if not resp:
    raise ValueError("Safety first! Close the curtains")

resp = get_yes_no("Ensure the laser is set to remote fire\n Is the laser set to remote?")

if not resp:
    raise ValueError("Computer will not fire laser unless set to remote")
else:
    logging.warning("WARNING: CHANGE TO START BEFORE OPENING CURTAIN FOR ANY REASON \n")

Please flip back in preparation for lysis. 
 Has the mirror been flipped up? y
Please shut the curtains surrounding the system. 
 Have the curtains been closed? y
Ensure the laser is set to remote fire
 Is the laser set to remote? y


## Test the Lysis Positions


In [264]:
import threading
prefix = f'center_{int(center*1e6):08d} x_{int(xy[0]*1e4):04d} y_{int(xy[1]*1e4):04d} '

def laser_fire(ce_system, *args):
    for i in range(5):
        logging.warning(f"WARNING: Laser will start firing in {5-i} s")
        time.sleep(.6)
    
    time.sleep(0.5)
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()
    time.sleep(1)
    print("Laser has Fired, Run Next cell")


# Move to the bright field position, fire the laser
print("Firing at position determined by bright field")
ce_system.objective.set_z(bf_climb_focus)
threading.Thread(target=laser_fire, args=(ce_system,)).start()

print("Preparing to fire...")

Firing at position determined by bright field


Preparing to fire...


Laser has Fired, Run Next cell


In [265]:
bf_auto_lysis = get_yes_no("Did Lysis Occur? Image will freeze until answered")
save_a_note(prefix, f"BF {bf_climb_focus}", bf_auto_lysis, output_csv)

#Move up slightly
ce_system.xy_stage.set_rel_y(0.003)

# Move to the fluorescence position, fire the laser
print("Firing at position determined by bright field")
ce_system.objective.set_z(fl_climb_focus)

threading.Thread(target=laser_fire, args=(ce_system,)).start()
print("Laser Fired, Run Next cell")

Did Lysis Occur? Image will freeze until answered y
Firing at position determined by bright field


NameError: name 'fl_climb_focus' is not defined

In [329]:
fl_auto_lysis = get_yes_no("Did Lysis Occur? Image will freeze until answered")
save_a_note(prefix, f"FL {fl_climb_focus}", fl_auto_lysis, output_csv)

if not bf_auto_lysis and not fl_auto_lysis:
    get_yes_no(" No cells were lysed. This will require you to adjust to focus manually and test the lysis. \n You must remain " 
               "at a position near (<10 um) the cell. Press y to continue and run the box below this")
else:
    
    print("Cell lysis was a success. Find a new cell and start over.")
    resp = get_yes_no("Please switch laser from remote to standby mode. \n Is the laser switched to off/standby?")
    while not resp:
        resp = get_yes_no("Please switch laser from remote to standby mode. \n Is the laser switched to off/standby?")
    print("It is now safe to lift the curtain")

Did Lysis Occur? Image will freeze until answered n
 No cells were lysed. This will require you to adjust to focus manually and test the lysis. 
 You must remain at a position near (<10 um) the cell. Press y to continue and run the box below this y


If the cell did not lyse, move the focus until a cavitation/cell lysis occurs. 
Run the following cell to lyse the cell. Best to keep firing remotely for consistency (it is easy not to press the fire button correctly). 

In [261]:
resp = get_yes_no("Fire the lysis laser?")
if resp:
    threading.Thread(target=laser_fire, args=(ce_system,)).start()
else:
    raise InterruptedError("Please rerun when you wish to fire the laser")
#print("Laser fired, run next cell")

Fire the lysis laser? y


Laser has Fired, Run Next cell


In [262]:
resp = get_yes_no("Did a cavitation occur?")
z = ce_system.objective.read_z()
save_a_note(prefix, f"Manual {z}", resp, output_csv)

if resp:
    print("Cell Lysis success, find a new cell and start over from the top. Removing laser from software standby.")
    resp = get_yes_no("Please switch laser from remote to start mode. \n Is the laser switched to off/start?")
    while not resp:
        resp = get_yes_no("Please switch laser from remote to start mode. \n Is the laser switched to off/start?")
    print("It is now safe to lift the curtain")
else:
    print("Repeat this code block until lysis/cavitation")

Did a cavitation occur? y
Cell Lysis success, find a new cell and start over from the top. Removing laser from software standby.
Please switch laser from remote to start mode. 
 Is the laser switched to off/start? y
It is now safe to lift the curtain
